# INCEPTION AND VGG

In [117]:
from PIL import Image
import cv2
import pytesseract
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import fitz
import os
import glob
import time
import img2pdf
import shutil
import keras
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

in_path = '/home/vimalkumar/Downloads/new_project (copy)/cnn validation/testing-5'


dic = {0:'Blank',1:'Not Blank'}

inception_model = keras.models.load_model('stewart_inception_model.h5')
vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')

file_name = []
inception_result = []
vgg_result = []

cnt = 1

for im in sorted(os.listdir(in_path)):
    #print(cnt)
    print(cnt,'  ',im)
    cnt = cnt + 1
    img = image.load_img(os.path.join(in_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = inception_model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    output_inception = dic[a[0]]
    #print(output_inception)
    file_name.append(im)
    
    image_read = cv2.imread(os.path.join(in_path,im),0)
    _, blackAndWhite = cv2.threshold(image_read, 127, 255, cv2.THRESH_BINARY_INV)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 50:   #filter small dotted regions
            img2[labels == i + 1] = 255

    res = cv2.bitwise_not(img2)
    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
    image_data_lower = " ".join(image_data.splitlines()).lower()
    
    if output_inception == 'Blank':
        inception_result.append(output_inception)
    else:
        
        
        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            inception_result.append('New Document')
        else:
            inception_result.append('Not Blank')
            
        
    
    val_1 = vgg_model.predict(images)
    #print(val_1)

    a_1=np.argmax(val_1,axis=1)
    output_vgg = dic[a_1[0]]
    #print(output_vgg)
    #file_name.append(im)
    #vgg_result.append(output_vgg)
    
    if output_vgg == 'Blank':
        vgg_result.append(output_vgg)
    else:
        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()
        
        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
            vgg_result.append('New Document')
        else:
            vgg_result.append('Not Blank')
            
    
            
df = pd.DataFrame({'File':file_name,'Inception_result':inception_result,
                   'VGG_Result':vgg_result})
#display(df)

df['Comparison'] = df['Inception_result'] == df['VGG_Result']
df['Comparison'] = df['Comparison'].replace(True,'Yes')
df['Comparison'] = df['Comparison'].replace(False,'No')

display(df)

df.to_excel('/home/vimalkumar/Downloads/new_project (copy)/cnn validation/cnn_5.xlsx')

end = time.time()

print(end - start)


1    101.jpeg
2    102.jpeg
3    103.jpeg
4    104.jpeg
5    105.jpeg
6    106.jpeg
7    107.jpeg
8    108.jpeg
9    109.jpeg
10    110.jpeg
11    111.jpeg
12    112.jpeg
13    113.jpeg
14    114.jpeg
15    115.jpeg
16    116.jpeg
17    117.jpeg
18    118.jpeg
19    119.jpeg
20    120.jpeg
21    121.jpeg
22    122.jpeg
23    123.jpeg
24    124.jpeg
25    125.jpeg
26    126.jpeg
27    127.jpeg
28    128.jpeg
29    129.jpeg
30    130.jpeg
31    131.jpeg
32    132.jpeg
33    133.jpeg
34    134.jpeg
35    135.jpeg
36    136.jpeg
37    137.jpeg
38    138.jpeg
39    139.jpeg
40    140.jpeg
41    141.jpeg
42    142.jpeg
43    143.jpeg
44    144.jpeg
45    145.jpeg
46    146.jpeg
47    147.jpeg
48    148.jpeg
49    149.jpeg
50    150.jpeg
51    151.jpeg
52    152.jpeg
53    153.jpeg
54    154.jpeg
55    155.jpeg
56    156.jpeg
57    157.jpeg
58    158.jpeg
59    159.jpeg
60    160.jpeg
61    161.jpeg
62    162.jpeg
63    163.jpeg
64    164.jpeg
65    165.jpeg
66    166.jpeg
67    167.jpeg
68  

,File,Inception_result,VGG_Result,Comparison
0,101.jpeg,Not Blank,Not Blank,Yes
1,102.jpeg,Not Blank,Not Blank,Yes
2,103.jpeg,Not Blank,Not Blank,Yes
3,104.jpeg,Not Blank,Not Blank,Yes
4,105.jpeg,Not Blank,Not Blank,Yes
5,106.jpeg,Not Blank,Not Blank,Yes
6,107.jpeg,Blank,Blank,Yes
7,108.jpeg,Not Blank,Not Blank,Yes
8,109.jpeg,Not Blank,Not Blank,Yes
9,110.jpeg,Not Blank,Not Blank,Yes


230.43581676483154


In [71]:
df['Comparison'].value_counts()

Yes    583
No      18
Name: Comparison, dtype: int64

In [118]:
df['Comparison'].value_counts()

Yes    160
No      11
Name: Comparison, dtype: int64

# Inception and Pytesseract

In [14]:
from PIL import Image
import cv2
import pytesseract
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import fitz
import os
import glob
import time
import img2pdf
import shutil
import keras
from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

in_path = '/home/vimalkumar/Downloads/new_project (copy)/cnn validation/test'


dic = {0:'Blank',1:'Not Blank'}

inception_model = keras.models.load_model('stewart_inception_model.h5')
vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')



file_name = []
inception_result = []
vgg_result = []
ocr_result = []


cnt = 1

for im in sorted(os.listdir(in_path)):
    #print(cnt)
    print(cnt,'  ',im)
    cnt = cnt + 1
    img = image.load_img(os.path.join(in_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = inception_model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    output_inception = dic[a[0]]
    #print(output_inception)
    file_name.append(im)
    
    image_read = cv2.imread(os.path.join(in_path,im),0)
    
   
    
    _, blackAndWhite = cv2.threshold(image_read, 127, 255, cv2.THRESH_BINARY_INV)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 50:   #filter small dotted regions
            img2[labels == i + 1] = 255

    res = cv2.bitwise_not(img2)
#     data = Image.fromarray(res)
#     enh_con = ImageEnhance.Contrast(data)
#     contrast = 3.0
#     image_contrasted = enh_con.enhance(contrast)
    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
    image_data_lower = " ".join(image_data.splitlines()).lower()
    
    image_data_1 = pytesseract.image_to_string(image_read,config=r'--oem 3 --psm 11')
    image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
    #print(image_data_1_lower)
    
    
    
    if output_inception == 'Blank':
        inception_result.append(output_inception)
    else:
        
        
        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            inception_result.append('New Document')
        else:
            inception_result.append('Not Blank')
            
            
    val_1 = vgg_model.predict(images)
    #print(val_1)

    a_1=np.argmax(val_1,axis=1)
    output_vgg = dic[a_1[0]]
    #print(output_vgg)
    #file_name.append(im)
    #vgg_result.append(output_vgg)
    
    if output_vgg == 'Blank':
        vgg_result.append(output_vgg)
    else:
        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()
        
        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
            vgg_result.append('New Document')
        else:
            vgg_result.append('Not Blank')
            
    
    
    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
    if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
        ocr_result.append('New Document')
        
    elif len(image_data_1_lower) < 15:
        ocr_result.append('Blank')
        
    else:
        ocr_result.append('Not Blank')
        
        
df = pd.DataFrame({'File':file_name,'Inception_result':inception_result,
                   'VGG_result':vgg_result,'OCR_Result':ocr_result})
#display(df)

df['Comparison'] = (df['Inception_result'] == df['VGG_result'])&(df['Inception_result'] == df['OCR_Result'])

df['Comparison'] = df['Comparison'].replace(True,'Yes')
df['Comparison'] = df['Comparison'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

df.to_excel('/home/vimalkumar/Downloads/new_project (copy)/cnn validation/cnn_4.xlsx')            

end = time.time()

print(end - start)


1    513.jpg
2    514.jpg
3    515.jpg
4    516.jpg
5    517.jpg
6    518.jpg
7    519.jpg
8    520.jpg
9    521.jpg
10    522.jpg
11    523.jpg
12    524.jpg
13    525.jpg
14    526.jpg
15    527.jpg
16    528.jpg
17    529.jpg
18    530.jpg
19    531.jpg
20    532.jpg
21    533.jpg
22    534.jpg
23    535.jpg
24    536.jpg


,File,Inception_result,VGG_result,OCR_Result,Comparison
0,513.jpg,New Document,New Document,New Document,Yes
1,514.jpg,Not Blank,Not Blank,Not Blank,Yes
2,515.jpg,Not Blank,Not Blank,Not Blank,Yes
3,516.jpg,Not Blank,Not Blank,Not Blank,Yes
4,517.jpg,New Document,New Document,New Document,Yes
5,518.jpg,Not Blank,Not Blank,Not Blank,Yes
6,519.jpg,Blank,Not Blank,Not Blank,No
7,520.jpg,New Document,New Document,New Document,Yes
8,521.jpg,Blank,Not Blank,Not Blank,No
9,522.jpg,Not Blank,Not Blank,Not Blank,Yes


56.72465896606445


In [2]:
df['Comparison'].value_counts()

Yes    180
No       9
Name: Comparison, dtype: int64

In [7]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

in_path = '/home/vimalkumar/Downloads/new_project (copy)/cnn validation/test'


dic = {0:'Blank',1:'Not Blank'}

inception_model = keras.models.load_model('stewart_inception_model.h5')
vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')



file_name = []
inception_result = []
vgg_result = []
ocr_result = []


cnt = 1

for im in sorted(os.listdir(in_path)):
    #print(cnt)
    print(cnt,'  ',im)
    cnt = cnt + 1
    img = image.load_img(os.path.join(in_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = inception_model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    output_inception = dic[a[0]]
    #print(output_inception)
    file_name.append(im)
    
    #image_read = cv2.imread(os.path.join(in_path,im),0)
    pil_read = Image.open(os.path.join(in_path,im))
    pil_read_gray = ImageOps.grayscale(pil_read)
    img_rgb = np.array(pil_read_gray)
   
    
    _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 50:   #filter small dotted regions
            img2[labels == i + 1] = 255

    res = cv2.bitwise_not(img2)
#     data = Image.fromarray(res)
#     enh_con = ImageEnhance.Contrast(data)
#     contrast = 3.0
#     image_contrasted = enh_con.enhance(contrast)
    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
    image_data_lower = " ".join(image_data.splitlines()).lower()
    
    stats = ImageStat.Stat(pil_read)
    #print(stats)
    
    y = []
    for band,name in enumerate(pil_read.getbands()):
        x=sum(stats.extrema[band])
        y.append(x)
    if sum(y)==765:
        #print('good')
        image_data_1 = pytesseract.image_to_string(pil_read_gray)
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
            
    else:
        #print('dark')
        image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
        #print(image_data_1_lower)
    
    
    
    if output_inception == 'Blank':
        inception_result.append(output_inception)
    else:
        
        
        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            inception_result.append('New Document')
        else:
            inception_result.append('Not Blank')
            
            
    val_1 = vgg_model.predict(images)
    #print(val_1)

    a_1=np.argmax(val_1,axis=1)
    output_vgg = dic[a_1[0]]
    #print(output_vgg)
    #file_name.append(im)
    #vgg_result.append(output_vgg)
    
    if output_vgg == 'Blank':
        vgg_result.append(output_vgg)
    else:
        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()
        
        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
            vgg_result.append('New Document')
        else:
            vgg_result.append('Not Blank')
            
    
    
    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
    if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
        ocr_result.append('New Document')
        
    elif len(image_data_1_lower) < 15:
        ocr_result.append('Blank')
        
    else:
        ocr_result.append('Not Blank')
        
        
df = pd.DataFrame({'File':file_name,'Inception_result':inception_result,
                   'VGG_result':vgg_result,'OCR_Result':ocr_result})
#display(df)

df['Comparison'] = (df['Inception_result'] == df['VGG_result'])&(df['Inception_result'] == df['OCR_Result'])

df['Comparison'] = df['Comparison'].replace(True,'Yes')
df['Comparison'] = df['Comparison'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

df.to_excel('/home/vimalkumar/Downloads/new_project (copy)/cnn validation/cnn_5.xlsx')            

end = time.time()

print(end - start)    

1    008.jpeg
2    009.jpeg
3    010.jpeg
4    011.jpeg
5    018.jpeg
6    020.jpeg
7    022.jpeg
8    030.jpeg
9    033.jpeg
10    034.jpeg
11    035.jpeg
12    036.jpeg
13    048.jpeg
14    056.jpeg
15    065.jpeg
16    068.jpeg
17    069.jpeg
18    073.jpeg
19    077.jpeg
20    082.jpeg
21    093.jpeg
22    100.jpeg
23    1039.jpeg
24    104.jpeg
25    1041.jpeg
26    1042.jpeg
27    1043.jpeg
28    1044.jpeg
29    1045.jpeg
30    1046.jpeg
31    1047.jpeg
32    1048.jpeg
33    1052.jpeg
34    1053.jpeg
35    1054.jpeg
36    1055.jpeg
37    1057.jpeg
38    1059.jpeg
39    1060.jpeg
40    1061.jpeg
41    1062.jpeg
42    1067.jpeg
43    1068.jpeg
44    1069.jpeg
45    107.jpeg
46    1070.jpeg
47    1071.jpeg
48    1072.jpeg
49    110.jpeg
50    114.jpeg
51    119.jpeg
52    123.jpeg
53    125.jpeg
54    128.jpeg
55    135.jpeg
56    141.jpeg
57    1424.jpeg
58    1425.jpeg
59    1426.jpeg
60    1427.jpeg
61    1428.jpeg
62    1429.jpeg
63    143.jpeg
64    1430.jpeg
65    1431.jpeg
66

OSError: image file is truncated (72 bytes not processed)

In [11]:
df['Comparison'].value_counts()

Yes    167
No     103
Name: Comparison, dtype: int64

In [ ]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

start = time.time()

in_path = '/home/vimalkumar/Downloads/new_project (copy)/cnn validation/test'


dic = {0:'Blank',1:'Not Blank'}

inception_model = keras.models.load_model('stewart_inception_model.h5')
vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')



file_name = []
inception_result = []
vgg_result = []
ocr_result = []


cnt = 1

for im in sorted(os.listdir(in_path)):
    #print(cnt)
    print(cnt,'  ',im)
    cnt = cnt + 1
    img = image.load_img(os.path.join(in_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = inception_model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    output_inception = dic[a[0]]
    #print(output_inception)
    file_name.append(im)
    
    #image_read = cv2.imread(os.path.join(in_path,im),0)
    pil_read = Image.open(os.path.join(in_path,im))
    pil_read_gray = ImageOps.grayscale(pil_read)
    img_rgb = np.array(pil_read_gray)
   
    
    _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 50:   #filter small dotted regions
            img2[labels == i + 1] = 255

    res = cv2.bitwise_not(img2)
#     data = Image.fromarray(res)
#     enh_con = ImageEnhance.Contrast(data)
#     contrast = 3.0
#     image_contrasted = enh_con.enhance(contrast)
    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
    image_data_lower = " ".join(image_data.splitlines()).lower()
    
    stats = ImageStat.Stat(pil_read)
    #print(stats)
    
    y = []
    for band,name in enumerate(pil_read.getbands()):
        x=sum(stats.extrema[band])
        y.append(x)
    if sum(y)==765:
        #print('good')
        image_data_1 = pytesseract.image_to_string(pil_read_gray)
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
            
    else:
        #print('dark')
        image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
        #print(image_data_1_lower)
    
    
    
    if output_inception == 'Blank':
        inception_result.append(output_inception)
    else:
        
        
        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            inception_result.append('New Document')
        else:
            inception_result.append('Not Blank')
            
            
    val_1 = vgg_model.predict(images)
    #print(val_1)

    a_1=np.argmax(val_1,axis=1)
    output_vgg = dic[a_1[0]]
    #print(output_vgg)
    #file_name.append(im)
    #vgg_result.append(output_vgg)
    
    if output_vgg == 'Blank':
        vgg_result.append(output_vgg)
    else:
        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()
        
        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
            vgg_result.append('New Document')
        else:
            vgg_result.append('Not Blank')
            
    
    
    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
    if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
        ocr_result.append('New Document')
        
    elif len(image_data_1_lower) < 15:
        ocr_result.append('Blank')
        
    else:
        ocr_result.append('Not Blank')
        
        
df = pd.DataFrame({'File':file_name,'Inception_result':inception_result,
                   'VGG_result':vgg_result,'OCR_Result':ocr_result})
#display(df)

df['Comparison'] = (df['Inception_result'] == df['VGG_result'])&(df['Inception_result'] == df['OCR_Result'])

df['Comparison'] = df['Comparison'].replace(True,'Yes')
df['Comparison'] = df['Comparison'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

df.to_excel('/home/vimalkumar/Downloads/new_project (copy)/cnn validation/cnn_5.xlsx')            

end = time.time()

print(end - start)    

# Updated version

# Single Pdf file

In [23]:
start = time.time()

pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'

file_path = '/home/vimalkumar/Documents/stewart/Pdf files'


in_path = os.path.join(file_path,pdf_file)
out_path = os.path.join('/home/vimalkumar/Documents/stewart/Images',
                        str(os.path.splitext(pdf_file)[0]))

os.makedirs(out_path)

doc = fitz.open(in_path)

for i in range(len(doc)):
    doc = fitz.open(in_path)
    page = doc.load_page(i)  # number of page
    pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
    file_name = str(i+1000 + 1)       
    pix.pil_save(os.path.join(out_path,file_name+'.jpeg'))
    
dic = {0:'Blank',1:'Not Blank'}

inception_model = keras.models.load_model('stewart_inception_model.h5')
vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')



file_name = []
inception_result = []
vgg_result = []
ocr_result = []


cnt = 1

for im in sorted(os.listdir(out_path)):
    #print(cnt)
    print(cnt,'  ',im)
    cnt = cnt + 1
    img = image.load_img(os.path.join(out_path,im),
                     target_size=(224,224))
    #plt.imshow(img)
    #plt.show()

    x = image.img_to_array(img)
    x = x/255
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = inception_model.predict(images)
    #print(val)

    a=np.argmax(val,axis=1)
    output_inception = dic[a[0]]
    #print(output_inception)
    file_name.append(im)
    
    #image_read = cv2.imread(os.path.join(in_path,im),0)
    pil_read = Image.open(os.path.join(out_path,im))
    pil_read_gray = ImageOps.grayscale(pil_read)
    img_rgb = np.array(pil_read_gray)
   
    
    _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 50:   #filter small dotted regions
            img2[labels == i + 1] = 255

    res = cv2.bitwise_not(img2)
#     data = Image.fromarray(res)
#     enh_con = ImageEnhance.Contrast(data)
#     contrast = 3.0
#     image_contrasted = enh_con.enhance(contrast)
    image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
    image_data_lower = " ".join(image_data.splitlines()).lower()
    
    stats = ImageStat.Stat(pil_read)
    #print(stats)
    
    y = []
    for band,name in enumerate(pil_read.getbands()):
        x=sum(stats.extrema[band])
        y.append(x)
    if sum(y)==765:
        #print('good')
        image_data_1 = pytesseract.image_to_string(pil_read_gray)
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
            
    else:
        #print('dark')
        image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
        image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
        image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
        #print(image_data_1_lower)
    
    
    
    if output_inception == 'Blank':
        inception_result.append(output_inception)
    else:
        
        
        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            inception_result.append('New Document')
        else:
            inception_result.append('Not Blank')
            
            
    val_1 = vgg_model.predict(images)
    #print(val_1)

    a_1=np.argmax(val_1,axis=1)
    output_vgg = dic[a_1[0]]
    #print(output_vgg)
    #file_name.append(im)
    #vgg_result.append(output_vgg)
    
    if output_vgg == 'Blank':
        vgg_result.append(output_vgg)
    else:
        #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
        #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
        #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()
        
        fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
        if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
            vgg_result.append('New Document')
        else:
            vgg_result.append('Not Blank')
            
    
    
    fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
    fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')
        
    if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
        ocr_result.append('New Document')
        
    elif len(image_data_1_lower) < 15:
        ocr_result.append('Blank')
        
    else:
        ocr_result.append('Not Blank')
        
        
df = pd.DataFrame({'File':file_name,'Inception_result':inception_result,
                   'VGG_result':vgg_result,'OCR_Result':ocr_result})
#display(df)

df['Comparison'] = (df['Inception_result'] == df['VGG_result'])&(df['Inception_result'] == df['OCR_Result'])

df['Comparison'] = df['Comparison'].replace(True,'Yes')
df['Comparison'] = df['Comparison'].replace(False,'No')

pd.set_option('display.max_rows',1000)
display(df)

df.to_excel(os.path.join(file_path,str(os.path.splitext(pdf_file)[0])+'.xlsx'))            

end = time.time()

print(end - start) 
    



1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg
25    1025.jpeg
26    1026.jpeg
27    1027.jpeg
28    1028.jpeg
29    1029.jpeg
30    1030.jpeg
31    1031.jpeg
32    1032.jpeg
33    1033.jpeg
34    1034.jpeg
35    1035.jpeg
36    1036.jpeg
37    1037.jpeg
38    1038.jpeg
39    1039.jpeg
40    1040.jpeg
41    1041.jpeg
42    1042.jpeg
43    1043.jpeg
44    1044.jpeg
45    1045.jpeg
46    1046.jpeg
47    1047.jpeg
48    1048.jpeg
49    1049.jpeg
50    1050.jpeg
51    1051.jpeg
52    1052.jpeg
53    1053.jpeg
54    1054.jpeg
55    1055.jpeg
56    1056.jpeg
57    1057.jpeg
58    1058.jpeg
59    1059.jpeg
60    1060.jpeg
61    1061.jpeg
62    1062.jpeg
63    1063.jpeg
6

,File,Inception_result,VGG_result,OCR_Result,Comparison
0,1001.jpeg,Not Blank,Not Blank,Not Blank,Yes
1,1002.jpeg,New Document,New Document,New Document,Yes
2,1003.jpeg,Not Blank,Not Blank,Not Blank,Yes
3,1004.jpeg,Not Blank,Not Blank,Not Blank,Yes
4,1005.jpeg,Not Blank,Not Blank,Not Blank,Yes
5,1006.jpeg,Not Blank,Not Blank,Not Blank,Yes
6,1007.jpeg,Not Blank,Not Blank,Not Blank,Yes
7,1008.jpeg,Not Blank,Not Blank,Not Blank,Yes
8,1009.jpeg,New Document,New Document,New Document,Yes
9,1010.jpeg,Not Blank,Not Blank,Not Blank,Yes


259.26325368881226


In [24]:
df['Comparison'].value_counts()

Yes    83
No      6
Name: Comparison, dtype: int64

# Mutliple Pdf file

In [1]:
from PIL import Image,ImageStat,ImageOps
import cv2
import pytesseract
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
import fitz
import os
import re
import glob
import time
import img2pdf
import shutil
import keras
from PIL import ImageEnhance
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt


start = time.time()

#pdf_file = 'Lindbergh Lawns Tract  BL 1,2,3.pdf'

file_path = '/home/vimalkumar/Documents/stewart/Pdf files'
excel_path = '/home/vimalkumar/Documents/stewart/excel_output'

for pdf_file in sorted(os.listdir(file_path)):
    print(pdf_file)
    in_path = os.path.join(file_path,pdf_file)
    out_path = os.path.join('/home/vimalkumar/Documents/stewart/Images',
                            str(os.path.splitext(pdf_file)[0]))

    os.makedirs(out_path)

    doc = fitz.open(in_path)

    for i in range(len(doc)):
        doc = fitz.open(in_path)
        page = doc.load_page(i)  # number of page
        pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
        file_name = str(i+1000 + 1)       
        pix.pil_save(os.path.join(out_path,file_name+'.jpeg'))

    dic = {0:'Blank',1:'Not Blank'}

    inception_model = keras.models.load_model('stewart_inception_model.h5')
    vgg_model = keras.models.load_model('stewart_vgg16_model_1.h5')


    pdf_name = []
    file_name = []
    inception_result = []
    vgg_result = []
    ocr_result = []


    cnt = 1

    for im in sorted(os.listdir(out_path)):
        pdf_name.append(pdf_file)
        #print(cnt)
        print(cnt,'  ',im)
        cnt = cnt + 1
        img = image.load_img(os.path.join(out_path,im),
                         target_size=(224,224))
        #plt.imshow(img)
        #plt.show()

        x = image.img_to_array(img)
        x = x/255
        x = np.expand_dims(x,axis=0)
        images = np.vstack([x])
        val = inception_model.predict(images)
        #print(val)

        a=np.argmax(val,axis=1)
        output_inception = dic[a[0]]
        #print(output_inception)
        file_name.append(im)

        #image_read = cv2.imread(os.path.join(in_path,im),0)
        pil_read = Image.open(os.path.join(out_path,im))
        pil_read_gray = ImageOps.grayscale(pil_read)
        img_rgb = np.array(pil_read_gray)


        _, blackAndWhite = cv2.threshold(img_rgb, 127, 255, cv2.THRESH_BINARY_INV)

        nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(blackAndWhite, None, None, None, 8, cv2.CV_32S)
        sizes = stats[1:, -1] #get CC_STAT_AREA component
        img2 = np.zeros((labels.shape), np.uint8)

        for i in range(0, nlabels - 1):
            if sizes[i] >= 50:   #filter small dotted regions
                img2[labels == i + 1] = 255

        res = cv2.bitwise_not(img2)
    #     data = Image.fromarray(res)
    #     enh_con = ImageEnhance.Contrast(data)
    #     contrast = 3.0
    #     image_contrasted = enh_con.enhance(contrast)
        image_data = pytesseract.image_to_string(res,config=r'--oem 3 --psm 11')
        image_data_lower = " ".join(image_data.splitlines()).lower()

        stats = ImageStat.Stat(pil_read)
        #print(stats)

        y = []
        for band,name in enumerate(pil_read.getbands()):
            x=sum(stats.extrema[band])
            y.append(x)
        if sum(y)==765:
            #print('good')
            image_data_1 = pytesseract.image_to_string(pil_read_gray)
            image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
            image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)

        else:
            #print('dark')
            image_data_1 = pytesseract.image_to_string(pil_read_gray,config=r'--oem 3 --psm 11')
            image_data_1_lower = " ".join(image_data_1.splitlines()).lower()
            image_data_1_lower = re.sub(r"\s+",'', image_data_1_lower)
            #print(image_data_1_lower)



        if output_inception == 'Blank':
            inception_result.append(output_inception)
        else:


            fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
            fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

            if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
                inception_result.append('New Document')
            else:
                inception_result.append('Not Blank')


        val_1 = vgg_model.predict(images)
        #print(val_1)

        a_1=np.argmax(val_1,axis=1)
        output_vgg = dic[a_1[0]]
        #print(output_vgg)
        #file_name.append(im)
        #vgg_result.append(output_vgg)

        if output_vgg == 'Blank':
            vgg_result.append(output_vgg)
        else:
            #image_read_1 = cv2.imread(os.path.join(in_path,im),0)
            #image_data_1 = pytesseract.image_to_string(image_read_1,config=r'--oem 3 --psm 6')
            #image_data_lower_1 = " ".join(image_data_1.splitlines()).lower()

            fuzz_ratio_vgg = fuzz.partial_ratio(image_data_lower,'new document to follow')
            fuzz_ratio_vgg_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

            if fuzz_ratio_vgg >= 80 or fuzz_ratio_vgg_1 >= 80:
                vgg_result.append('New Document')
            else:
                vgg_result.append('Not Blank')



        fuzz_ratio = fuzz.partial_ratio(image_data_lower,'new document to follow')
        fuzz_ratio_1 = fuzz.partial_ratio(image_data_lower,'new file to follow')

        if fuzz_ratio >= 80 or fuzz_ratio_1 >= 80:
            ocr_result.append('New Document')

        elif len(image_data_1_lower) < 15:
            ocr_result.append('Blank')

        else:
            ocr_result.append('Not Blank')


    df = pd.DataFrame({'PDF':pdf_name,'File':file_name,'Inception_result':inception_result,
                       'VGG_result':vgg_result,'OCR_Result':ocr_result})
    #display(df)

    df['Comparison'] = (df['Inception_result'] == df['VGG_result'])&(df['Inception_result'] == df['OCR_Result'])

    df['Comparison'] = df['Comparison'].replace(True,'Yes')
    df['Comparison'] = df['Comparison'].replace(False,'No')

    pd.set_option('display.max_rows',1000)
    display(df)

    df.to_excel(os.path.join(excel_path,str(os.path.splitext(pdf_file)[0])+'.xlsx'))            

end = time.time()

print(end - start) 
    



/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2022-02-10 11:44:12.517085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-10 11:44:12.517129: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


1507-1513.pdf


2022-02-10 11:44:52.865981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-10 11:44:52.866022: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-10 11:44:52.866046: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-02-10 11:44:52.866610: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1    1001.jpeg


2022-02-10 11:45:02.026665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


2    1002.jpeg
3    1003.jpeg
4    1004.jpeg


,PDF,File,Inception_result,VGG_result,OCR_Result,Comparison
0,1507-1513.pdf,1001.jpeg,New Document,New Document,New Document,Yes
1,1507-1513.pdf,1002.jpeg,Not Blank,Not Blank,Not Blank,Yes
2,1507-1513.pdf,1003.jpeg,Not Blank,Not Blank,Not Blank,Yes
3,1507-1513.pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Yes


Batch 17 (1).pdf
1    1001.jpeg
2    1002.jpeg
3    1003.jpeg
4    1004.jpeg
5    1005.jpeg
6    1006.jpeg
7    1007.jpeg
8    1008.jpeg
9    1009.jpeg
10    1010.jpeg
11    1011.jpeg
12    1012.jpeg
13    1013.jpeg
14    1014.jpeg
15    1015.jpeg
16    1016.jpeg
17    1017.jpeg
18    1018.jpeg
19    1019.jpeg
20    1020.jpeg
21    1021.jpeg
22    1022.jpeg
23    1023.jpeg
24    1024.jpeg


,PDF,File,Inception_result,VGG_result,OCR_Result,Comparison
0,Batch 17 (1).pdf,1001.jpeg,Not Blank,Not Blank,Not Blank,Yes
1,Batch 17 (1).pdf,1002.jpeg,Not Blank,Not Blank,Not Blank,Yes
2,Batch 17 (1).pdf,1003.jpeg,Not Blank,Not Blank,Not Blank,Yes
3,Batch 17 (1).pdf,1004.jpeg,Not Blank,Not Blank,Not Blank,Yes
4,Batch 17 (1).pdf,1005.jpeg,Not Blank,Not Blank,Not Blank,Yes
5,Batch 17 (1).pdf,1006.jpeg,Not Blank,Not Blank,Not Blank,Yes
6,Batch 17 (1).pdf,1007.jpeg,Not Blank,Not Blank,Not Blank,Yes
7,Batch 17 (1).pdf,1008.jpeg,Not Blank,Not Blank,Not Blank,Yes
8,Batch 17 (1).pdf,1009.jpeg,Not Blank,Not Blank,Not Blank,Yes
9,Batch 17 (1).pdf,1010.jpeg,Not Blank,Not Blank,Not Blank,Yes


77.65412521362305


In [9]:
df['Comparison'].value_counts()

Yes    83
No      6
Name: Comparison, dtype: int64